In [1]:
# CÉLULA 1

# Se precisar, descomente e execute esta linha para instalar as dependências
# !pip install langchain langchain-groq langchain-pinecone sentence-transformers torch python-dotenv ipywidgets

import os
from dotenv import load_dotenv
from operator import itemgetter
from IPython.display import display, Markdown

# Imports do LangChain
from langchain_groq import ChatGroq
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_pinecone import PineconeVectorStore
from langchain.prompts import PromptTemplate
from langchain_core.runnables import RunnableBranch, RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

print("Bibliotecas importadas com sucesso!")

Bibliotecas importadas com sucesso!


/Users/iagocampista/anaconda3/envs/travelAgent/lib/python3.11/site-packages/langchain_pinecone/__init__.py:3: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  from langchain_pinecone.vectorstores import Pinecone, PineconeVectorStore


In [2]:
# CÉLULA 2

# --- Carregar Variáveis de Ambiente ---
load_dotenv()

# --- Inicialização do LLM e Retriever ---
llm = ChatGroq(model_name="llama-3.3-70b-versatile", temperature=0)
model_name = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
# Inicializar o Retriever para buscar informações no Pinecone
index_name = "assistente-viagem"
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2",
    model_kwargs={'device': 'cpu'}
)
vectorstore = PineconeVectorStore(index_name=index_name, embedding=embeddings)
retriever = vectorstore.as_retriever()

# --- Definição dos Prompts ---
itinerary_prompt = PromptTemplate(    
    template="""Você é um planejador de viagens experiente.
    Com base na PERGUNTA do usuário e no CONTEXTO de informações turísticas, crie um roteiro de viagem detalhado.
    O roteiro deve ser claro, organizado por dias e períodos (manhã, tarde, noite) e incluir sugestões do contexto.
    Se não for possível formular uma resposta com as informações do contexto, diga que você não sabe. Não invente.

    CONTEXTO: {context}
    PERGUNTA: {question}
    ROTEIRO DETALHADO:""",
    input_variables=["context", "question"],
    )

logistics_prompt = PromptTemplate(
    template="""Você é um especialista em logística de viagens.
    Responda à PERGUNTA do usuário focando em transporte, direções e aspectos práticos.
    Use o CONTEXTO para basear sua resposta.
    Se a informação não estiver no contexto, diga que você não sabe. Não invente.

    CONTEXTO: {context}
    PERGUNTA: {question}
    RESPOSTA:""",
    input_variables=["context", "question"],
)

local_info_prompt = PromptTemplate(
    template="""Você é um assistente de viagens muito prestativo.
    Use o CONTEXTO fornecido para responder à PERGUNTA do usuário.
    Se a informação não estiver no contexto, diga que você não sabe. Não invente.

    CONTEXTO: {context}
    PERGUNTA: {question}
    RESPOSTA:""",
    input_variables=["context", "question"],
) 
translation_prompt = PromptTemplate(
    template="""Você é um guia de tradução para turistas.
    O usuário quer saber frases úteis em outro idioma.
    Traduza a seguinte frase ou pedido para o idioma solicitado e forneça uma pronúncia simplificada.

    PEDIDO DO USUÁRIO: {question}
    TRADUÇÃO E GUIA DE PRONÚNCIA:""",
    input_variables=["question"]
) 
router_prompt_template = """Dada a pergunta do usuário, classifique-a em uma das seguintes categorias.
Responda APENAS com o nome da categoria.

Categorias:
- roteiro-viagem: O usuário está pedindo um itinerário, plano de viagem ou sugestões de atividades para múltiplos dias.
- logistica-transporte: O usuário está perguntando sobre meios de transporte, como chegar a um lugar, aeroportos ou locomoção.
- info-local: O usuário está pedindo informações específicas sobre um ponto turístico, restaurante, horário de funcionamento, etc.
- traducao-idiomas: O usuário está pedindo para traduzir uma frase ou palavra.
- geral: O usuário está fazendo uma pergunta geral que não se encaixa nas outras categorias.

Pergunta do usuário:
{question}

Classificação:"""

# --- Definição das Cadeias Especializadas ---
itinerary_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | itinerary_prompt | llm | StrOutputParser()
)
logistics_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | logistics_prompt | llm | StrOutputParser()
)
local_info_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | local_info_prompt | llm | StrOutputParser()
)
translation_chain = (
    # O itemgetter aqui garante que a entrada no formato de dicionário também funcione
    itemgetter("question") 
    | translation_prompt 
    | llm 
    | StrOutputParser()
)


# --- Definição do Roteador (Classifier + RunnableBranch) ---
#prompt_router = PromptTemplate(template=router_prompt_template, input_variables=["question"])
prompt_router = PromptTemplate(
    template=router_prompt_template,
    input_variables=["question"],
)
classifier_chain = prompt_router | llm | StrOutputParser()

branch = RunnableBranch(
    (lambda x: "roteiro-viagem" in x["topic"].lower(), itinerary_chain),
    (lambda x: "logistica-transporte" in x["topic"].lower(), logistics_chain),
    (lambda x: "info-local" in x["topic"].lower(), local_info_chain),
    (lambda x: "traducao-idiomas" in x["topic"].lower(), translation_chain),
    llm,
)

chain = {
    "topic": (lambda x: {"question": x["input"]}) | classifier_chain,
    "question": lambda x: x["input"]
} | branch

print("Assistente pronto para começar! Execute a próxima célula para iniciar o chat.")


/var/folders/3x/j5lxxqwd671dhlhn57rtnkyr0000gn/T/ipykernel_39723/1638079777.py:11: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(


Assistente pronto para começar! Execute a próxima célula para iniciar o chat.


In [3]:
# CÉLULA 3

def iniciar_chat():
    display(Markdown("### 🤖 Olá! Sou seu assistente de viagens."))
    display(Markdown("**Digite sua pergunta abaixo ou 'sair' para encerrar.**"))
    print("-" * 60)

    while True:
        try:
            query = input("Você: ")

            if query.lower() in ["sair", "exit", "fim"]:
                display(Markdown("**🤖 Até a próxima!**"))
                break
            
            if not query.strip():
                continue

            # Roda o classificador para descobrir a rota e exibe
            topic = classifier_chain.invoke({"question": query})
            display(Markdown(f">>> _Roteador acionou a chain: **{topic}**_"))

            # Roda a cadeia principal para obter a resposta
            response = chain.invoke({"input": query})
            
            # Exibe a resposta
            display(Markdown(f"**Assistente:** {response}"))
            print("-" * 60)

        except KeyboardInterrupt:
            # Permite interromper o loop com Ctrl+C ou pelo botão "Stop" do Jupyter
            display(Markdown("\n**🤖 Chat encerrado.**"))
            break
        except Exception as e:
            display(Markdown(f"**Ocorreu um erro:** {e}"))
            print("-" * 60)

# Inicia o chat
iniciar_chat()

### 🤖 Olá! Sou seu assistente de viagens.

**Digite sua pergunta abaixo ou 'sair' para encerrar.**

------------------------------------------------------------


**🤖 Até a próxima!**